In [95]:
import pandas as pd
import numpy as np
from astropy.time import Time
import glob
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit


In [96]:
times = np.array([])
price = np.array([])

files = glob.glob('*.csv')

files.sort()

for file in files:
    df = pd.read_csv(file)
    print(df.columns)
    
    data = df.to_numpy()
    print(data[0])
    price = np.concatenate((price,data[:,1]))
    times = np.concatenate((times,data[:,0]))



JD_times = []
for i,time in enumerate(times):
    time = time.split(' - ')[0]
    
    year = time[6:10]
    day = time[:2]
    month = time[3:5]
    hour = time[-5:-2]

    time = year+'-'+month+'-'+day+'T'+hour+'00:00'
    JD_times.append(time)

print(JD_times[0])
test_times = ['1999-01-01T00:00:00.123456789', '2010-01-01T00:00:00']
t = Time(JD_times, format='isot', scale='utc')
t.jd

Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|DK2'], dtype='object')
['01.01.2020 00:00 - 01.01.2020 01:00' 33.42 'EUR' nan]
Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|DK2'], dtype='object')
['01.01.2021 00:00 - 01.01.2021 01:00' 50.87 'EUR' nan]
Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|DK2'], dtype='object')
['01.01.2022 00:00 - 01.01.2022 01:00' 46.6 'EUR' nan]
Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|DK2'], dtype='object')
['01.01.2023 00:00 - 01.01.2023 01:00' 2.01 'EUR' nan]
Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|DK2'], dtype='object')
['01.01.2024 00:00 - 01.01.2024 01:00' '29.13' 'EUR' nan]
2020-01-01T00:00:00


array([2458849.5       , 2458849.54166667, 2458849.58333333, ...,
       2460676.375     , 2460676.41666667, 2460676.45833333],
      shape=(43853,))

In [97]:
%matplotlib tk
fig, ax = plt.subplots()

index = np.where(price != 'n/e')
t = t[index]
price = price[index].astype('float')

index_nan = np.where(np.isnan(price)==False)
price = price[index_nan]
t = t[index_nan]


ax.scatter(t.jd,price)
plt.show()

In [4]:
N = len(price)
I = abs(fft(price))

F = fftfreq(N, 1/24)

F = fftfreq(N, 1/24)[:N//2]

In [5]:
%matplotlib tk
fig, ax = plt.subplots()

ax.plot(F, np.abs(I[0:N//2])**2)
ax.set_yscale('log')
plt.show()

In [71]:
# Function for normalizing the prize time series.
# This will erase information on long scale. For day ahead trading we are however mostly interested in 
#the short term fluctuations. i.e. what will happen on the scale of 1 day.

def moving_median(P,R):
    '''
    R: timeframe that we consider [int in hours]
    P: Price [np.array]
    '''
    new_P = np.zeros(len(P))
    N = int(len(P)/R) #Number of frames
    for i in range(N):
        i = i * R
        j = i + R
        new_P[i:j] = P[i:j] - np.median(P[i:j])
    return new_P


new_price = moving_median(price,12)


def sine_fit(P,R,w=1,A=100, phi=0,B=0,C=0):
    '''
    R: timeframe that we consider [int in hours]
    P: Price [np.array]
    w: frequency of fluctuation in range R in days
        Set to 1 for obvious reasons  
    '''
    def sine_line(t,A,B,C,w,phi):
        return A*np.sin(w*t+phi) + B*t + C

    p_init = [A,B,C,w,phi]

    new_P = np.zeros(len(P))
    N = int(len(P)/R) #Number of frames
    
    for i in range(N):
        i = i * R
        j = i + R
        time = np.arange(0,len(P[i:j]))
        popt, pcov = curve_fit(sine_line, time, P[i:j],p0=p_init,absolute_sigma=True)

        
        new_P[i:j] = sine_line(time,*popt)

        
    return new_P


def line_fit(P,R,A=0, B=0,):
    '''
    R: timeframe that we consider [int in hours]
    P: Price [np.array]
    w: frequency of fluctuation in range R in days
        Set to 1 for obvious reasons  
    '''
    def line(t,A,B):
        return A*t + B

    p_init = [A,B]

    new_P = np.zeros(len(P))
    N = int(len(P)/R) #Number of frames
    
    for i in range(N):
        i = i * R
        j = i + R
        time = np.arange(0,len(P[i:j]))
        popt, pcov = curve_fit(line, time, P[i:j],p0=p_init,absolute_sigma=True)

        
        new_P[i:j] = line(time,*popt)

        
    return new_P

    


In [73]:
%matplotlib tk
fig, ax = plt.subplots()

new_p = line_fit(price,R=48,B=0,A=0)

ax.plot(t.jd,new_p)
ax.scatter(t.jd,price)
plt.show()

In [23]:
N = len(new_price)
I = abs(fft(new_price))

F = fftfreq(N, 1/24)

F = fftfreq(N, 1/24)[:N//2]

In [25]:
%matplotlib tk
fig, ax = plt.subplots()

ax.plot(F, np.abs(I[0:N//2])**2)
#ax.set_yscale('log')
plt.show()

In [27]:
from functions import daylength

In [47]:
days = np.arange(0,365)
length = np.array([daylength(day,-90) for day in days])


In [48]:
%matplotlib tk
fig, ax = plt.subplots()

ax.scatter(days,length)
plt.show()

In [58]:
def get_limited_time(time,price,L,R=1,lat=56):
    '''
    Function for getting the points where length of day is limited to a range
    t: time in Julian data (np.array)
    P: price (np.array)
    L: length of day in hours (float)
    R: range of length of day in hours (float)
    lat: lattitude (float)

    Returns the times and prices where days are within a specific length L+/-R/2 
    '''
    #Julian date has 0 at -4375-01-01T12:00:00
    print(time)
    mod_time = time%365
    lengths = np.array([daylength(day,lat) for day in mod_time])

    time_0 = time[0]

    index = np.where( abs(lengths-L)<R/2)

    return time[index], price[index]


    

In [99]:
new_time, new_price = get_limited_time(t.jd,price,12,1,56)
print(len(new_price))
N = len(new_price)
I = abs(fft(new_price))

F = fftfreq(N, 1/24)

F = fftfreq(N, 1/24)[:N//2]

%matplotlib tk
fig, ax = plt.subplots()

ax.plot(F, np.abs(I[0:N//2])**2)
ax.set_yscale('log')
plt.show()


[2458849.5        2458849.54166667 2458849.58333333 ... 2460588.375
 2460588.41666667 2460588.45833333]
3010


In [93]:

def moving_sine(t,w1=np.pi,):
    if t%np.pi*2<np.pi:
        return np.sin(w1*t)
    if t%np.pi*2>=np.pi:
        return np.sin(w1*0.9*t)


%matplotlib tk
fig, ax = plt.subplots()

t = np.linspace(0,np.pi*4,1000)
ax.plot(t,[moving_sine(x) for x in t])
plt.show()


In [117]:

xs = np.linspace(0,40*np.pi,1000)
S = [np.sin(x*np.pi*5) if x<np.median(xs) else  np.sin(x*np.pi*3)+10 for x in xs ]

N = len(S)
I = abs(fft(S))

F = fftfreq(N, 1/24)

F = fftfreq(N, 1/24)[:N//2]

%matplotlib tk
fig, ax = plt.subplots()

ax.plot(F, np.abs(I[0:N//2])**2)

plt.show()

In [112]:
%matplotlib tk
fig, ax = plt.subplots()

ax.plot(xs,S)

plt.show()